# Build an Multi-agentic Services with Orchestration for Reasoning

This workshop will focus on creating a sophisticated system of coordinated AI agents. We'll incorporate recent breakthroughs in generative AI to enhance the system's reasoning capabilities. By leveraging the power of LangGraph and the advanced language models available through Amazon Bedrock, including Claude-3.5 and LLaMA-3.1, we will create a versatile and capable system that can tackle a wide range of challenges.

<img src="./images/agentic_integration.png" width="950" />

Key Features:

* Agentic Service: The agent will be designed as a service, allowing for seamless integration and deployment in various applications.
* Dynamic Prompt Rewriting: The agent will dynamically rewrite prompts to optimize the responses from the underlying language models, ensuring more accurate and informative outputs.
* Adaptive Routing: Inspired by the Semantic Router, the routing agent will intelligently route requests to retrieval, web search, or pre-trained LLMs for the most desirable answers, leveraging the strengths of each method for optimal performance. This adaptive routing mechanism will ensure that the agent can effectively handle a diverse set of queries and tasks.
* Hallucination Grader: The agent will include a hallucination grader component to assess the reliability of the generated responses. This will help identify and correct any hallucinations or incomplete answers.
* Human Involvement: If needed, the agent will be able to involve human subject matter experts to provide additional verification and correction of the responses, further improving the trustworthiness and reliability of the system.

By combining these advanced reasoning techniques, the agentic services with orchestration will be able to provide more accurate and informative responses to challenging queries. This will be a significant step forward in the development of intelligent agents that can truly understand and respond to complex questions.

To build this powerful system, we will use LangGraph to create complex, multi-step workflows that involve language models and other components. This will allow us to develop a flexible and scalable system that can handle a wide range of tasks.

To further enhance our capabilities, we will port the original notebook to utilize Amazon Bedrock for LLM inference. This will enable us to leverage the cloud processing power and take advantage of the advanced language models available through Amazon Bedrock, such as Claude-3 and LLaMA-3. By harnessing the power of these cutting-edge language models, we will be able to push the boundaries of what is possible with intelligent agents.

While the choice of vector stores (local chromaDB) will remain unchanged for now, we will explore how to scale this part in future blog posts, ensuring that our system can handle ever-growing amounts of data and information.

Join us in this exciting workshop as we embark on a journey to create an intelligent agent that redefines the boundaries of what is possible with language-based AI systems. Together, we will explore the latest advancements in the field and push the limits of what can be achieved.



In [ ]:
#!pip install -r ../requirements.txt  -U 
!pip install crewai[tools]

## 1. Setting Up API keys or tokens 

To access various services, such as Amazon Bedrock for Large Language Models (LLMs) and embedding models, Tavily web search engine, and optional Langchain, you will need to set up and obtain the necessary API keys or tokens. These API keys and tokens serve as authentication credentials that allow your application to securely connect and interact with the respective services.

In [ ]:
import sys
import os
import boto3
import json
import requests


aws_region = "us-west-2" # choose your region you operate in
os.environ['TAVILY_API_KEY'] = tavily_ai_api_key = 'tvly-NA' # For extra search result. Optional
os.environ['OPENAI_API_KEY'] = openai_api_key = 'sk-NA' # Only when you elect to use OpenAI's ada as embedding model. Otherwise you just need to assign an empty key. 
# Temp image file
temp_gen_image = "./delme.png"
markdown_filename = "./blogpost.md"

module_paths = ["./", "../scripts"]
for module_path in module_paths:
    sys.path.append(os.path.abspath(module_path))

from blog_writer import *
from bedrock import *

#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

## 2. Creating a Bedrock Runtime Client
We'll create a Bedrock runtime client to connect to the Amazon Bedrock service. Bedrock, a fully managed service by AWS, allows developers to build and deploy generative AI models like large language models (LLMs). This client will enable us to leverage pre-trained LLMs from Amazon, such as the powerful LLaMA3 model from Meta.

Connecting to Bedrock is crucial for building our scalable and secure RAG agent, as it provides the necessary language model for generation capabilities. With the Bedrock runtime client in place, we can integrate LLaMA3 into our workflow and use its advanced natural language processing capabilities to generate accurate responses.

In [ ]:
### Select models
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

options = ["mistral.mistral-large-2407-v1:0", "anthropic.claude-3-haiku-20240307-v1:0", "anthropic.claude-3-5-sonnet-20240620-v1:0", "meta.llama3-1-70b-instruct-v1:0"]
# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    value=options[1],
    description='Choose an option:',
    disabled=False,
)

# Display the dropdown widget
display(dropdown)

In [ ]:
model_id = dropdown.value
llm = get_llm(model_id)
model_id_l31 = 'meta.llama3-1-70b-instruct-v1:0'
model_id_c35 = model_id_c35 = "anthropic.claude-3-sonnet-20240229-v1:0" # Due to model access restriction #'anthropic.claude-3-5-sonnet-20240620-v1:0' 
model_id_mistral_large = 'mistral.mistral-large-2407-v1:0'
# Choose multiple models for different purpose to deversify and avoid potential bias 
llm = get_llm(model_id)
llm_llama31 = get_llm(model_id_l31)
llm_claude35 = get_llm(model_id_c35 )
llm_mistral = get_llm(model_id_mistral_large)

## 3. Create agentic services with multi-agent capability

Creating agentic services with multi-agent capability using Amazon Bedrock, Converse API, and LangChain can be a powerful approach to building intelligent and collaborative systems. Amazon Bedrock provides a foundation for developing large language models (LLMs) and integrating them into applications, while the Converse API enables seamless communication between these models and external services. LangChain, on the other hand, offers a framework for building complex, multi-agent systems that can leverage the capabilities of various LLMs and other AI components. By combining these tools, developers can create agentic services that can engage in dynamic, context-aware interactions, share knowledge, and coordinate their efforts to tackle complex tasks. This approach can be particularly useful in scenarios where a diverse set of specialized agents need to collaborate, such as in enterprise automation, customer service, or research and development.

In [ ]:
from IPython.display import Image, Markdown
from langgraph.prebuilt import create_react_agent
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain_aws import BedrockEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage, AnyMessage
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from botocore.exceptions import ClientError
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults

In [ ]:
config = Config(
        retries = dict(
            max_attempts = 10,
            total_max_attempts = 25,
        )
    )
bedrock_client = boto3.client("bedrock-runtime", config=config) 

In [ ]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, client, region_name: str, model_id: str):
        self.embedder = BedrockEmbeddings(
            client=client,
            region_name=region_name,
            model_id=model_id
        )
    def embed_query(self, query: str) -> Embeddings:
        return self.embedder.embed_query(query)
    def embed_documents(self, documents: list[str]) -> Embeddings:
        return self.embedder.embed_documents(documents)

class MultiAgentState(TypedDict):
    question: str
    question_type: str
    answer: str
    feedback: str


memory = MemorySaver()
embedding_model_id = "amazon.titan-embed-text-v2:0"

####
# Router
###
def route_question(state: MultiAgentState):
    print('route function execution')
    print(state)
    return state['question_type']


####
# rewrite the question
####
def rewrite_node(state: MultiAgentState):
    """
   REwrite question from query to match domain expert
    Args:
        question (str): The user query
    Returns:
        promt (str): rewrite question to form an expert prompt
    """
    print("---REWRITE QUESTION---")
    c3_template = """Rewrite the question by following the {{instruction}} to capture more precise and comprehensive intent from {question}.
    <instructions> 
        <step>Identify the key purposes, concepts and entities in the original {{question}}.</step> 
        <step>Rephrase the question to be more specific and focused, ensuring that the language is clear and unambiguous.</step> 
        <step>Provide additional context or background information that may be helpful for web search or RAG system to better understand and respond to the question.</step> 
        <step>Output your reqritten question only without answering it or repeating the riginal one.</step>
    </instructions> 
    """
    
    c3_prompt = ChatPromptTemplate.from_template(c3_template)
    #chain = ( c3_prompt | llm_c3 | StrOutputParser() | (lambda x: x.split("\n")))
    rewritten_chain = ( c3_prompt | llm | StrOutputParser() )
    rewritten_question = rewritten_chain.invoke({"question": state['question']})
    print(rewritten_question)
    if os.path.exists(temp_gen_image):
        os.remove(temp_gen_image)
    return {"answer": rewritten_question}

    
#####
# Router agent
#####
question_category_prompt = '''You are a senior specialist of analytical support. Your task is to classify the incoming questions. 
Depending on your answer, question will be routed to the right team, so your task is crucial for our team. 
There are 5 possible question types: 
- Vectorstore - Answer questions related to pre-indexed healthcare and medical research related topics stored in the vactorestore.
- Websearch- Answer questions based on events happened recently, after most LLM's cut-off dates. 
- General - Answer questions for LLM or a few LLMs.
- Text2image - Generate an image from text input.
- Booking - Assist in restaurant reservation booking.
- BlogWriter - Writer a blog post about the provided topic as a professional writer.
Return in the output only one word (VECTORSTORE, WEBSEARCH, GENERAL, TEXT2IMAGE, BOOKING or BLOGWRITER).
'''

def router_node(state: MultiAgentState):
    print('Router node started execution')
    messages = [
        SystemMessage(content=question_category_prompt), 
        HumanMessage(content=state['question'])
    ]
    response = llm.invoke(messages)
    print('Question type: %s' % response.content)
    return {"question_type": response.content}


#####
# Search agent
#####
def search_expert_node(state: MultiAgentState):
    tavily_tool = TavilySearchResults(max_results=5)
    duck_search = DuckDuckGoSearchResults()

    search_expert_system_prompt = '''
    You are an expert in LangChain and other technologies. 
    Your goal is to answer questions based on results provided by search.
    You don't add anything yourself and provide only information baked by other sources. 
    '''
    search_agent = create_react_agent(llm, [duck_search, tavily_tool],
        state_modifier = search_expert_system_prompt)
    messages = [HumanMessage(content=state['question'])]
    result = search_agent.invoke({"messages": messages})
    return {'answer': result['messages'][-1].content}


#######
# RAG
#######
def rag_node(state: MultiAgentState):
    urls = [
        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11127599/",
        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11127585/",
        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11127581/"
    ]
    c3_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. Use  less than 10 sentences maximum and keep the answer concise. 
    
    {context} 
    
    Use these to craft an answer to the question: {question}"""
    c3_prompt = ChatPromptTemplate.from_template(c3_template)
    
    docs = [WebBaseLoader(url).load() for url in urls]    
    docs_list = [item for sublist in docs for item in sublist]
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=4000, chunk_overlap=400
    )
    doc_splits = text_splitter.split_documents(docs_list)

    embedding_function = MyEmbeddingFunction(client = bedrock_client,
                                             region_name=aws_region,
                                             model_id=embedding_model_id)
    # Add to vectorDB
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        embedding=embedding_function,
        collection_name="rag-chroma-titan-embed-text-v2-1",
    )
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 3, 'lambda_mult': 0.25})
    rag_chain = c3_prompt | llm | StrOutputParser()
    documents = retriever.invoke(state['question'])
    generation = rag_chain.invoke({"context": documents, "question": state['question']})
    #generation = rag_chain.invoke({"context": documents, "question": state['answer']}) # Use the rewritten question instead
    return {'answer': generation}


#####
# LLM node
####
def llm_node(state: MultiAgentState):
    model_ids = [model_id_mistral_large , model_id_l31]
    max_tokens = 2048
    temperature = 0.01
    top_p = 0.95

    conversation = [
        {
            "role": "user",
            #"system": "You are a domain expert who can understand the intent of user query and answer question truthful and professionally. Please, don't provide any unchecked information and just tell that you don't know if you don't have enough info.",
            "content": [{"text": state['question']}],
        }
    ]
    try:
        # Send the message to the model, using a basic inference configuration.
        responses = []
        for model_id in model_ids:
            response = bedrock_client.converse(
                modelId=model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": max_tokens, "temperature": temperature, "topP": top_p},
            )
        
            # Extract and print the response text.
            responses.append( response["output"]["message"]["content"][0]["text"])

        ###
        # Combine the answers to form a unified one
        ###
        c3_template = """Your are a domain expert and your goal is to Merge and eliminate redundant elements from {{responses}} that captures the essence of all input while adhering to the following the {{instruction}}.
        <instructions> 
            <step>Aggregate relevant information from the provided context.</step> 
            <step>Eliminate redundancies to ensure a concise response.</step> 
            <step>Maintain fidelity to the original content.</step> 
            <step>Add additional relevent info to the question or removing iirelevant information.</step>
        </instructions> 
        <responses>
            {responses}
        </responses>
        """
        
        messages = [
            SystemMessage(content=c3_template), 
            HumanMessage(content=state['question'])
        ]

        return {'answer': llm_claude35.invoke(messages)}
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")


####
# Human in the loop
###

def human_feedback_node(state: MultiAgentState):
    editor_prompt = '''You're an editor and your goal is to provide the final answer to the customer, taking into account the feedback. 
    You don't add any information on your own. You use friendly and professional tone. 
    In the output please provide the final answer to the customer without additional comments.
    Here's all the information you need.
    
    
    Question from customer: 
    ----
    {question}
    ----
    Draft answer:
    ----
    {answer}
    ----
    Feedback: 
    ----
    {feedback}
    ----
    '''
    print(state)
    messages = [
        SystemMessage(content=editor_prompt.format(question = state['question'], answer = state['answer'], feedback = state['feedback']))
    ]
    response = llm.invoke(messages)
    return {"answer": response.content}

def editor_node(state: MultiAgentState):
    pass
    print(state)
    messages = [
        SystemMessage(content=editor_prompt.format(question = state['question'], answer = state['answer'], feedback = state['feedback']))
    ]
    response = llm.invoke(messages)
    return {"answer": response.content}

#####
# multi-agent collaboration node
#####
def blog_writer_node(state: MultiAgentState):
    blog_crew = blogCrew(topic=state['answer'], model_id=model_id_l31)
    result = blog_crew.run()

    ## Werite to a Markdown file
    if os.path.exists(markdown_filename):
        os.remove(markdown_filename)
    # Create the Markdown format and Save the Markdown text to a file
    markdown_text = f"# Sample Text\n\n{result.raw}\n\n![Image]({temp_gen_image})"
    with open(markdown_filename, "w") as file:
        file.write(markdown_text)

    return {"answer": result}

#### Additional functions

In [ ]:
#####
# Hallucination grader
#####
from langchain.callbacks.base import BaseCallbackHandler
import random
import base64

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs):
        print(f"Response: {response}")
        
def hallucination_grader_node(state:MultiAgentState):
    c3_template = """You are a grader assessing whether an answer is grounded in supported by facts. 
        Give a binary score 'pass' or 'fail' score to indicate whether the answer is grounded in supported by a 
        set of facts in your best knowledge. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
        
        Here is the answer: {answer}"""
    c3_prompt = ChatPromptTemplate.from_template(c3_template)
    
    # Grade by a diff model in this case Claude 3
    #hallucination_grader = prompt | llm_llama31  | JsonOutputParser() 
    hallucination_grader = c3_prompt | llm_claude35 | JsonOutputParser()
    score = hallucination_grader.invoke({"answer": state['answer'], "callbacks": [MyCustomHandler()]})
    return {'answer': score}

def hallucination_grader(state:MultiAgentState):
    c3_template = """You are a grader assessing whether an answer is grounded in supported by facts. 
        Give a binary score 'pass' or 'fail' score to indicate whether the answer is grounded in supported by a 
        set of facts in your best knowledge. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
        
        Here is the answer: {answer}"""
    c3_prompt = ChatPromptTemplate.from_template(c3_template)
    
    # Grade by a diff model in this case Claude 3
    #hallucination_grader = prompt | llm_llama31  | JsonOutputParser() 
    hallucination_grader = c3_prompt | llm_claude35 | JsonOutputParser()
    score = hallucination_grader.invoke({"answer": state['answer'], "callbacks": [MyCustomHandler()]})
    if "yes" in score['score'].lower():
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: the answer does not seem to contain hallucination ---"
        )
        return "END"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: the answer migh contain hallucination, next off to human review ---")
        return "to_human"


####
# Extra function but not as a node
####
def decide_to_search(state:MultiAgentState):
    """
    Determines whether to generate an answer, or add web search
    Args:
        state (dict): The current graph state
    Returns:
        str: Binary decision for next node to call
    """
    l31_prompt = PromptTemplate(
        template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
        an {answer} is grounded in / relevant to the {question}. Give a binary score 'yes' or 'no' score to indicate
        whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
        single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here is the answer:
        {answer}
        Here is the question: {question}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["question", "answer"],
    )
    
    answer_grader = l31_prompt | llm_llama31 | JsonOutputParser()
    print("---ASSESS GRADED ANSWER AGAINST QUESTION---")
    relevance = answer_grader.invoke({"answer": state["answer"], "question": state["question"]})
    print(relevance)
    if "yes" in relevance['score'].lower():
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: the answer is relevant to the question so it's ready for human review ---"
        )
        return "to_human"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: the answer is NOT relevant to the question then try web search ---")
        return "do_search"

#####
# text 2 image generation
####
def t2i_node2(state:MultiAgentState):
    negative_prompts = [
                "poorly rendered",
                "poor background details",
                "poorly drawn objects",
                "poorly focused objects",
                "disfigured object features",
                "cartoon",]
    body = json.dumps(
        {
            "taskType": "TEXT_IMAGE",
            "textToImageParams": {
                #"text":state['answer'].replace("{Rewritten Question}:\n\n", "")[:510],   # Required, Titan image gen v2 limits up to 512 token input
                "text":state['question'][:511],
                "negativeText": "poorly rendere, disfigured object features" #negative_prompts  # Optional
            },
            "imageGenerationConfig": {
                "numberOfImages": 1,   # Range: 1 to 5 
                "quality": 'premium',  # Options: standard or premium
                "height": 1024,         # Supported height list in the docs 
                "width": 1024,         # Supported width list in the docs
                "cfgScale": 6.5,       # Range: 1.0 (exclusive) to 10.0
                "seed": random.randint(1, 214783647)             # Range: 1 to 214783647
            }
        }
    )


    if os.path.exists(temp_gen_image):
        os.remove(temp_gen_image)
    response = bedrock_client.invoke_model(
        body=body, 
        modelId="amazon.titan-image-generator-v2:0",
        accept="application/json", 
        contentType="application/json"
    )
    response_body = json.loads(response["body"].read())
    with open(temp_gen_image, 'wb') as file:
        # Decode the base64 data and write it to the file
        file.write(base64.b64decode(response_body["images"][0]))
    return {"answer": temp_gen_image}

def t2i_node(state:MultiAgentState):
    url = "http://video.cavatar.info:8080/generate?prompt="
    prompt = f"Generate a high resiliution, photo realistic picture of {state['question']} with vivid color and attending to details."
    response = requests.get(url+prompt)
    if response.status_code == 200:
        image_bytes = response.content
    else:
        print(f"Error fetching image from {url}")
        pass
    with open(temp_gen_image, 'wb') as file:
        file.write(image_bytes)
    return {"answer": temp_gen_image}

### Pre-requisite: complete Bedrock agent association with knowledgeBase and lambda function to interact with pre-populated DynamoDB. OBtain the agent ID and Agent 

**Please note the next cell might take a few (> 10) minutes to complete**

You might use AWS console (https://us-west-2.console.aws.amazon.com/aos/home?region=us-west-2#opensearch/get-started-serverless) or aws cli to check the status
* %aws bedrock-agent list-agents
* %aws bedrock-agent list-agent-aliases --agent-id \<agent_id>
* %aws bedrock-agent list-knowledge-bases
* %aws bedrock-agent list-agent-knowledge-bases

In [ ]:
#%run ./create-agent-with-knowledge-base-and-action-group.ipynb

### Upon successful complettion of the Amazon Bedrock agent creation, define a node to invoke the agent.  

In [ ]:
####
# Bedrock agent integration
####
import uuid
import logging
from datetime import datetime
%store -r agent_id
%store -r alias_id

def invoke_BR_agent(agent_id, alias_id, query, enable_trace=False, session_state=dict()):
    session_id = str(uuid.uuid1())
    end_session = False
    logger = logging.getLogger(__name__)
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

def bedrock_agent_node(state:MultiAgentState):
    today = datetime.today().strftime('%b-%d-%Y')
    session_state = {
        "promptSessionAttributes": {
            "name": "John Doe",
            "today": today
        }
    }
    return {'answer': invoke_BR_agent(agent_id, alias_id, state["question"])}

## 4. Defining the Reasoning Flow with LangGraph Nodes and Edges

Implement nodes representing key actions: document retrieval, document grading, web search, and answer generation. Define conditional edges for decision-making: route the question, decide on document relevance, and grade the generated answer. Set up the workflow graph with entry points, nodes, and edges to ensure a logical progression through the RAG agent's steps. LangGraph allows us to define a graph-based workflow for our RAG agent, integrating document retrieval, question routing, answer generation, and self-correction into an efficient pipeline.

Key steps include:

* Question rewrite: Rewrite the query for better intend classification
* Routing: Deciding whether the question should go to the RAG, LLMs or a web search.
* Hallucination Grading: Ensuring the generated answer is grounded in the retrieved documents.
* Human in the loop: In case the answer fall bwloew desired quality, insert human feedback

LangGraph lets us seamlessly integrate these steps into a modular, adaptable workflow, enhancing the agent's ability to handle diverse queries.

In [ ]:
orch = StateGraph(MultiAgentState)
orch.add_node("rewrite", rewrite_node)
orch.add_node("router", router_node)
orch.add_node('search_expert', search_expert_node)
orch.add_node('healthcare_expert', rag_node)
orch.add_node('general_assistant', llm_node)
orch.add_node('text2image_generation', t2i_node)
orch.add_node('booking_assistant', bedrock_agent_node)
orch.add_node('blog_writer', blog_writer_node)
#orch.add_node('hallucination_grader', hallucination_grader_node)
orch.add_node('human', human_feedback_node)
#orch.add_node('editor', editor_node)

orch.add_conditional_edges(
    "router", 
    route_question,
    {'VECTORSTORE': 'healthcare_expert', 'WEBSEARCH': 'search_expert', 'GENERAL': 'general_assistant', 
     'TEXT2IMAGE': 'text2image_generation','BOOKING': 'booking_assistant', 'BLOGWRITER':'blog_writer'}
)

orch.set_entry_point("rewrite")
orch.add_edge('rewrite', 'router')
#orch.add_edge('healthcare_expert', 'human')
orch.add_conditional_edges(
    "healthcare_expert",
    decide_to_search,
    {
        "to_human": "human",
        "do_search": "search_expert",
    },
)
#orch.add_edge('search_expert', 'human')
orch.add_conditional_edges(
    "search_expert",
    decide_to_search,
    {
        "to_human": "human",
        "do_search": "search_expert",
    },
)
#orch.add_edge('general_assistant', 'hallucination_grader')
orch.add_edge('booking_assistant', END)
#orch.add_edge('hallucination_grader', 'human')
orch.add_conditional_edges(
    "general_assistant",
    hallucination_grader,
    {
        "to_human": "human",
        "END": END,
    },
)
orch.add_edge('human', END)
#orch.add_edge('editor', END)
orch.add_edge('blog_writer', 'text2image_generation')
orch.add_edge('text2image_generation', END)

## 5. Display the orchestration flows

The orchestration flows can be depicted using the following  visual representation that illustrate the sequence of operations, the data transformations, and the control flow between the different modules or algorithms involved in the vision comprehension process. By providing a clear and concise visual representation of the orchestration, it becomes easier for developers, researchers, and stakeholders to understand the overall architecture, identify potential bottlenecks or optimization opportunities, and communicate the system's functionality and performance.

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod #, NodeColors

graph = orch.compile(checkpointer=memory, interrupt_before = ['human'])
display(Image(graph.get_graph().draw_mermaid_png(
    curve_style=CurveStyle.LINEAR,
    #node_colors=NodeColors(start="#ffdfba", end="#baffc9", other="#fad7de"),
    #node_styles=custom_node_style,
    wrap_label_n_words=9,
    output_file_path=None,
    draw_method=MermaidDrawMethod.API,
    background_color="white",
    padding=20,
)))

## 6. Execute this orchestration pipeline with query driven reasoning  

Executing agentic services with multi-agent capability on executing a pipeline with query-driven reasoning and reactions involves the development of a system that can autonomously perform tasks and make decisions based on the information it gathers and the queries it receives. This system would consist of multiple intelligent agents, each with its own set of capabilities and knowledge, working together to achieve a common goal. The agents would use query-driven reasoning to understand the user's intent and then react accordingly, executing the necessary steps in the pipeline to provide the desired outcome. This approach allows for a more dynamic and adaptive system that can handle a wide range of tasks and respond to changing conditions in real-time. The result is a powerful and flexible service that can assist users with a variety of needs, from information retrieval to complex problem-solving.

In [ ]:
from PIL import Image

thread = {"configurable": {"thread_id": "42",  "recursion_limit": 5}}
results = []
prompts =[
        "Under what circumstances a patient should be screened for ectopic ACTH syndrome(EAS)?", # Use native RAG then human review if needed
        "What could be the typical clinical symptoms of Blepharitis?", # First try native RAG but not found then try Web search hen human review if needed
        "How many total medals did the US Olympic Team won in Paris 2024?", # Use Web search hen human review if needed
        "Why Steve Jobs was considered a legent in the tech world?", # Combine the answers from 2 LLMs then human review if needed
        "Generate a high res image of a colorful macaw reasting on tree, with vivid color and attending to details.",  # Use text-2-image generation 
        "Hi, I want to create a booking for 2 people, at 8pm on the 5th of May 2024.", #Use Bedrock agent to interact with KnowledgeBase and DynamoDB
        "Write a blog post about the 2024 uncrewed return of the Starliner space capsule pending safety concerns and helium leaks. State hoe NASA plan to return the two stranded astronauts." #Blog writting using CrewAI
        ]

for prompt in prompts:
    for event in graph.stream({'question':prompt,}, thread):
        print(event)
        results.append(event)
        if os.path.exists(temp_gen_image):
            Image.open(temp_gen_image).show()
    print("\n\n---------------------------------------\n\n")

#### (Optional) Display the generated blog

In [ ]:
from IPython.display import Markdown
with open(markdown_filename, 'r') as file:
    readme_content = file.read()

# Display the contents as Markdown
display(Markdown(readme_content))

### Next Steps:

1. Planning
2. Colaborative multi-agent reasoning
3. Momeory for multi-round and personalize reasoning
4. While this simple search-strategy shows a meaningful improvement in the success rate, it still struggles on long horizon tasks due to sparsity of environment rewards.
5. To combine a planning and reasoning agent with MCTS inference-time search and AI self-critique for self-supervised data collection, which we then use for RL type training.

## 4. Clean-up¶
Let's delete all the associated resources created to avoid unnecessary costs. Please change the markdown to code before encuring the cells

In [ ]:
%store -r table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions, alias_id, agent_id, agent_name
clean_up_resources(
    table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, kb_id, alias_id
)

In [ ]:
# Delete the agent roles and policies
delete_agent_roles_and_policies(agent_name)

In [ ]:
# delete KB
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)